# Get started with DownClim

`DownClim` is a simple and easy to use Python package for performing a climate downscaling on future climatic projections. Based on a reference product, and a set of projections from [CMIP6](https://pcmdi.llnl.gov/CMIP6/) and [CORDEX](https://cordex.org/) simulations, `DownClim` will help you to generate a climate projection for your region of interest.

Let's see it in action!

## The `DownClimContext` object 

In [2]:
from __future__ import annotations

import ee

from downclim.dataset.cmip6 import CMIP6Context
from downclim import (
    DownClimContext,
    define_DownClimContext_from_file,
    generate_DownClimContext_template_file,
)


In [1]:
from importlib_resources import files

from downclim import get_logger

logger = get_logger("downclim")
logger.setLevel("ERROR")

2025-08-29 13:58:01,621 - downclim - INFO - DownClim starting... Enjoy!
2025-08-29 13:58:01,623 - downclim - INFO - DownClim logging system initialized


In [3]:
your_ee_project_id = "downclim"
your_DownClimContext_yaml = files("downclim").parent.parent.joinpath("data", "DownClimContext_example.yaml")


In [4]:
# Authenticate to Earth Engine to retrieve CMIP6 data
# ee.Authenticate() if necessary, only need to do this once on your machine
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project=your_ee_project_id)

The `DownClimContext` object is the main object of the `DownClim` package. It contains all the relevant information necessary to perform your downscaling on your area of interest. 



You can check directly the documentation of the `DownClimContext` object to have an extensive overview of how to define it properly. In the following section, we will see the main fields included in the `DownClimContext` object to perform a downscaling.

### Main fields of the `DownClimContext` object

`DownClimContext` objects contain multiple attributes. The most relevant ones are:

- `aois`: the areas of interest. You can define one or multiple areas of interest, and you have the possibility to define them either as a `str`, a `geopandas.GeoDataFrame` or a simple Python `tuple` of coordinates.
    - `str`: the name provided must be recognized by [GADM](https://gadm.org/) to retrieve the corresponding shapefile.
    - `geopandas.GeoDataFrame` containing the directly shapefile of the area of interest.
    - `tuple` of coordinates `(lon_min, lon_max, lat_min, lat_max, str)` where the last `str` is the name of the area of interest.
  
- `variables`: the variables to downscale. You want to select the climate variables that you want to downscale in your future climate. You must provide them as a list of strings, and the name of the variables have to match the `CMIP6` / `CORDEX` naming convention (see [CMIP6 CMOR Tables](https://github.com/PCMDI/cmip6-cmor-tables)). For example, you can select the following variables: `['tas', 'pr']` for 2m temperature and precipitation. Please note that the variables selected must be available both in the climate simulations and in the reference product.

- `baseline_product`: the reference product used for downscaling. 

- `downscaling_methods`: the downscaling method to use. To downscale your climate data on your area of interest, you might want to use different downscaling methods. So far, `DownClim` only supports the `bias_correction` method, but other methods can be easily adopted and will be available soon.

- `use_cmip6` & `use_cordex`: whether to use CMIP6 or CORDEX simulations. You can select the climate simulation you want to use for your future projections. `CMIP6` projections are available for a large number of models and scenarios, at global scale and usually at a coarse resolution. `CORDEX` simulations are regional projections, much less numerous but at a finer resolution.

- `baseline_periods`, `evaluation_periods`, `projection_periods`: the years to use for the baseline, evaluation and projection periods. To perform a downscaling, you must define the baseline period, evaluation period and projection period. 
  - The baseline period is the period used to calibrate the downscaling method. It is a historical period that must overlap the reference product and the historical climate simulations.
  - The evaluation period is the period that overlaps the reference product and the future climate simulations. It is used to evaluate the calibration of the downscaling method performed on the baseline period. 
  - The projection period is the period for which you want to generate the future climate projections downscaled.


You can create a `DownClimContext` object either directly, or by defining a configuration file in a `.yaml` format.

### `DownClimContext` object creation directly

You can instantiate a `DownClimContext` object directly by providing the necessary information either via the constructor or providing a dictionary filled with the required keys. Not all fields are mandatory, and default values are used for the omitted fields.

In [5]:
DownClimContext_example = DownClimContext(
    aoi="Vanuatu",
    variable=["tas", "pr"],
    historical_period=(1980, 1981),
    evaluation_period=(2017, 2018),
    projection_period=(2099, 2100),
    use_cordex=False,
    use_cmip6=True,
    cmip6_context=CMIP6Context()
)


### `DownClimContext` object creation from a `.yaml` file

You can also define a `DownClimContext` object from a `.yaml` file. You must first create a template `.yaml` file. 

In [6]:
generate_DownClimContext_template_file(output_file = './DownClimContext_example.yaml')

And after filling the sections of the file, your `DownClim` configuration file should look like this:

```yaml
##
## All fields except aoi have default values. If the field is not specified, the default value is used.
####################################################
# aoi
aoi: "Vanuatu"

# variables
variable: ["tas", "pr"] 
time_frequency: "mon" 

# downscaling
downscaling_aggregation: "monthly-mean" 
downscaling_method: "bias_correction" 

# data and simulations
baseline_product: "chelsa2"
use_cordex: false 
use_cmip6: true 
cordex_context: {}
cmip6_context: { source: ["IPSL-CM6A-LR"] }
historical_period: [1980, 1981]
evaluation_period: [2017, 2018]
projection_period: [2099, 2100]
evaluation_product: ["chirps", "chelsa2"]

# internal computation
nb_threads: 4
memory_mb: 4096
chunks: { "time": 1, "lat": 1000, "lon": 1000 }

# directories
output_dir: "results"
tmp_dir: "tmp"
keep_tmp_dir: true

# data access
esgf_credential: "config/esgf_credential.yaml"
```

In [6]:
DownClimContext_example = define_DownClimContext_from_file(your_DownClimContext_yaml)
DownClimContext_example.model_dump()

{'aoi': [                                            geometry GID_0   NAME_0
  0  MULTIPOLYGON (((169.7766 -20.2488, 169.7719 -2...   VUT  Vanuatu],
 'variable': ['tas', 'pr'],
 'time_frequency': <Frequency.MONTHLY: 'monthly'>,
 'downscaling_aggregation': <Aggregation.MONTHLY_MEAN: 'monthly-mean'>,
 'baseline_product': <DataProduct.CHELSA: product_name='chelsa', period=(1979, 2018), scale_factor={'pr': 0.1, 'tas': 0.1, 'tasmin': 0.1, 'tasmax': 0.1}, add_offset={'pr': 0, 'tas': -273.15, 'tasmin': -273.15, 'tasmax': -273.15}, lon_lat_names={'lon': 'x', 'lat': 'y'}, url='https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL'>,
 'evaluation_product': [<DataProduct.CHIRPS: product_name='chirps', period=(1980, 2024), scale_factor={'pr': 1}, add_offset={'pr': 0}, lon_lat_names={'lon': 'lon', 'lat': 'lat'}, url='UCSB-CHG/CHIRPS/DAILY'>,
  <DataProduct.CHELSA: product_name='chelsa', period=(1979, 2018), scale_factor={'pr': 0.1, 'tas': 0.1, 'tasmin': 0.1, 'tasmax': 0.1}, add_offset={'pr': 0, 'tas': -2

## Download required data

Once your `DownClimContext` object is defined correctly, you can download the required data defined in the context. You can do it by using the `download_data` method of the `DownClimContext` object.

In [7]:
DownClimContext_example.download_data()

/Users/arsouze/miniconda3/envs/downclim/lib/python3.13/site-packages/rioxarray/crs.py:38: RuntimeWarning: CRS EPSG:4362 is deprecated. Its non-deprecated replacement EPSG:4956 will be used instead. To use the original CRS, set the OSR_USE_NON_DEPRECATED configuration option to NO.
  return rasterio.crs.CRS.from_user_input(crs_input)


## Perform a downscaling

Once your `DownClimContext` object is defined correctly and your data downloaded, you can directly perform a downscaling according to the context.

It gets as simple as: 

In [8]:
DownClimContext_example.run_downscaling()

## Evaluate your downscaling

You can even easily evaluate your downscaling over an evaluation period (as defined [above](#main-fields-of-the-downclimcontext-object)).

In [9]:
DownClimContext_example.run_evaluation(
    evaluation_grid_file = ["./results/chirps/chirps_Vanuatu_grid.nc", "./results/chelsa/chelsa_Vanuatu_grid.nc"]
)

As `results` has been used as the default output directory, you can find all files created there and your file tree should look like this:

```bash
.
├── aois
│   ├── Vanuatu.cpg
│   ├── Vanuatu.dbf
│   ├── Vanuatu.prj
│   ├── Vanuatu.shp
│   └── Vanuatu.shx
├── chelsa
│   ├── chelsa_Vanuatu_grid.nc
│   ├── Vanuatu_chelsa_monthly-mean_1980-1981.nc
│   └── Vanuatu_chelsa_monthly-mean_2017-2018.nc
├── chirps
│   ├── chirps_Vanuatu_grid.nc
│   └── Vanuatu_chirps_monthly-mean_2017-2018.nc
├── cmip6
│   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31.nc
│   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31.nc
│   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2099-01-01_2100-12-31.nc
├── downscaled
│   ├── cmip6
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_historical_r1i1p1f1_monthly-mean_1980-01-01_1981-12-31-chelsa_Vanuatu_grid.nc
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid.nc
│   │   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2099-01-01_2100-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid.nc
│   └── cordex
├── evaluation
│   ├── cmip6
│   │   ├── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chelsa_Vanuatu_grid_evaluation.nc
│   │   └── Vanuatu_cmip6_IPSL_IPSL-CM6A-LR_ssp245_r1i1p1f1_monthly-mean_2017-01-01_2018-12-31-downscaled-chelsa_baseline-chirps_Vanuatu_grid_evaluation.nc
│   └── cordex
└── regridder
    └── regridder-chelsa_Vanuatu_grid-to-chirps_Vanuatu_grid.nc
```

- `aois` : all files related to the areas of interest (AOIs) 
- `chelsa` : all files related to the CHELSA product. Contains the grid file (`netcdf` with `lon` and `lat` variables), in addition to climatologies for baseline and evaluation periods.
- `chirps` : all files related to the CHIRPS product. Contains the grid file (`netcdf` with `lon` and `lat` variables). As this product is used for evaluation only, it contains only a climatology over the evaluation period.
- `cmip6` : all files related to the CMIP6 product. All CMIP6 simulation files over the AOI matching the `CMIP6Context` defined in the `DownClimContext`.
- `cordex` : all files related to the CORDEX product. All CORDEX simulation files over the AOI matching the `CORDEXContext` defined in the `DownClimContext`. Here, empty as we did not requested any CORDEX simulations.
- `downscaled` : all files `CMIP6` and `CORDEX` downscaled products (over the baseline product). We performed 2 downscaling over 2 different grids, although only for the evaluation period for CHIRPS grid.
- `evaluation` : all files related to the evaluation of the downscaled products. 
- `regridder` : all files related to the regridding process. This process is internal to `DownClim`, but it can save a lot of time to reuse existing regridders among different grids.

## You're all set!

Now you can play with your `DownClimContext` object to explore, download, downscale and evaluate climate data over your areas of interest.

However, if you're looking for a finer grain of control over the dataset, we encourage you to [explore all DownClim functions](examples/all_functions.ipynb).